# Choosing features using simulated annealing

A quick experiment to see if it's possible to select features using a random walk.

The idea is simple, try out sets of feature by taking a random walk through the space of all possible features.  At each step, we toggle one feature on or off at random, and if our estimator performs better using this set of features we continue onwards from there.

If the estimator performs worse, we _probably_ go back to the previous step and try again, but with a small probability — depending on just how much worse — we choose this new set of features anyway.  This helps us avoid getting stuck in a local minimum.

To make sure we still settle with a good set of features, we decrease the probability of accepting a worse state a little each step.

Now, this is going to be a bit slow if we use our regular estimators for this competion, so just as in many other feature-selection techniques, it's probably better to use some very fast estimator that still can give us an idea of how important various features are.

For more details see the [repo](https://github.com/chrka/rwfs).

In [ ]:
!pip install git+https://github.com/chrka/rwfs.git

## Create dataset

We'll build a dataset from some of the hopefully more important features.  For simplicity, let's just use per-play features, and not any of the per-player features.

### Preprocessing

In [ ]:
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

np.random.seed(20191113)

Handle inconsistent team abbreviations.

In [ ]:
def _rename_team(df, fro, to):
    df.loc[df['VisitorTeamAbbr'] == fro, 'VisitorTeamAbbr'] = to
    df.loc[df['HomeTeamAbbr'] == fro, 'HomeTeamAbbr'] = to

    
class TeamAbbrCleaner(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        df = X.copy()
        _rename_team(df, 'BAL', 'BLT')
        _rename_team(df, 'CLE', 'CLV')
        _rename_team(df, 'ARI', 'ARZ')
        _rename_team(df, 'HOU', 'HST')
        return df

Create raw feature set (per-play data only).

In [ ]:
class FeaturePreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        df = X.groupby('PlayId').first().reset_index().copy()
        
        # Rename unchanged variables
        df.rename(columns={
            'GameId': 'game_id',
            'PlayId': 'play_id',
            'Season': 'season',
            'Quarter': 'quarter',
            'Down': 'down',
            'Distance': 'distance',
            'Week': 'week',
            'OffenseFormation': 'offense_formation',
            'DefendersInTheBox': 'defenders_in_the_box'
        }, inplace=True)
        
        # Arrange features from the offense's direction
        team_on_offense = np.where(df['PossessionTeam'] == df['HomeTeamAbbr'],
                                  'home', 
                                   'away')
        df['offense_is_home'] = team_on_offense == 'home'

        df['offense_score'] = np.where(df['offense_is_home'],
                                      df['HomeScoreBeforePlay'],
                                      df['VisitorScoreBeforePlay'])

        df['defense_score'] = np.where(df['offense_is_home'],
                                       df['VisitorScoreBeforePlay'],
                                       df['HomeScoreBeforePlay'])
        
        # This works even at YardLine 50 when FieldPosition is NA
        df['line_of_scrimmage'] = np.where(df['FieldPosition'] == df['PossessionTeam'],
                                           df['YardLine'],
                                           100 - df['YardLine'])

        # Time between snap and handoff
        time_handoff = pd.to_datetime(df['TimeHandoff'])
        time_snap = pd.to_datetime(df['TimeSnap'])
        time_to_handoff = (time_handoff - time_snap).dt.total_seconds()
        time_to_handoff = np.round(time_to_handoff).astype(int)
        df['time_to_handoff'] = time_to_handoff
        
        # Convert game clock to seconds
        game_clock = df['GameClock'].str.extract(r'(?P<MM>\d\d):(?P<SS>\d\d):\d\d')
        df['game_clock'] = 60 * game_clock['MM'].astype(int) + game_clock['SS'].astype(int)
        
        return df[['game_id', 'play_id', 'season', 'quarter', 'down', 'distance', 'week',
                   'offense_formation', 'defenders_in_the_box', 
                   'offense_is_home', 'offense_score', 'defense_score', 
                   'line_of_scrimmage', 'time_to_handoff', 'game_clock']]

In [ ]:
class TargetPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X.groupby('PlayId').first().reset_index()['Yards'].copy()

### Base feature set

Make everything numeric and reasonably scaled in order to be able to feed the data into a linear regressor.

In [ ]:
# ColumnTransformer does the same job, but loses column names.
# Adding them afterwords is quicker than this, but harder to maintain.
# A more convenient option is to use a name-preservering substitute or roll
# one's own.

def _hot_names(features, hot):
    names = []
    for feature, categories in zip(features, hot.categories_):
        names.extend([f"{feature}_{cat}" for cat in categories])
    return names

class PerPlayDatasetTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        # One-hot encode 'quarter' and 'down'
        self.quarter_down_one_hot = OneHotEncoder(handle_unknown='ignore', sparse=False)
        self.quarter_down_one_hot.fit(X[['quarter', 'down']])
        
        # Impute and one-hot encode 'offense_formation'
        self.encode_formation = Pipeline([
                ('imp', SimpleImputer(strategy='constant', fill_value='MISSING')),
                ('hot', OneHotEncoder(handle_unknown='ignore', sparse=False))
            ])
        self.encode_formation.fit(X[['offense_formation']])
        
        # One-hot encode 'time_to_handoff' for values of 1 and 2
        self.encode_tth = OneHotEncoder(categories=[[1, 2]], 
                                            handle_unknown='ignore',
                                            sparse=False)
        self.encode_tth.fit(X[['time_to_handoff']])

        # Min-max scale
        self.mm_scale = MinMaxScaler()
        self.mm_scale.fit(X[['distance', 'week', 'offense_score', 'defense_score', 
                             'line_of_scrimmage', 'game_clock']])

        self.scale_defenders_in_the_box = Pipeline([
            ('imp', SimpleImputer(strategy='most_frequent')),
            ('std', StandardScaler())
        ])
        self.scale_defenders_in_the_box.fit(X[['defenders_in_the_box']])
        
        return self
    
    def transform(self, X, y=None):
        # Passthrough values
        keep = X[['offense_is_home']]

        # One-hot encode 'quarter' and 'down'
        qd = self.quarter_down_one_hot.transform(X[['quarter', 'down']])
        qd = pd.DataFrame(qd, columns=_hot_names(['quarter', 'down'], self.quarter_down_one_hot))
        
        # Impute and one-hot encode 'offense_formation'
        fm = self.encode_formation.transform(X[['offense_formation']])
        fm = pd.DataFrame(fm, columns=_hot_names(['offense_formation'], 
                                                self.encode_formation.named_steps['hot']))
        
        # One-hot encode 'time_to_handoff' for values of 1 and 2
        tth = self.encode_tth.transform(X[['time_to_handoff']])
        tth = pd.DataFrame(tth, columns=_hot_names(['time_to_handoff'],
                                                  self.encode_tth))
        
        # Min-max scale
        mms = self.mm_scale.transform(X[['distance', 'week', 'offense_score', 'defense_score', 
                             'line_of_scrimmage', 'game_clock']])
        mms = pd.DataFrame(mms, columns=['distance', 'week', 'offense_score', 'defense_score', 
                           'line_of_scrimmage', 'game_clock'])
        
        ditb = self.scale_defenders_in_the_box.transform(X[['defenders_in_the_box']])
        ditb = pd.DataFrame(ditb, columns=['defenders_in_the_box'])
        
        return pd.concat([keep, qd, fm, tth, mms, ditb], axis=1)

In [ ]:
dataset_pipeline = Pipeline([
    ('clean', TeamAbbrCleaner()),
    ('features', FeaturePreprocessor()),
    ('play', PerPlayDatasetTransformer())
])

### Load dataset

In [ ]:
raw_df = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2020/train.csv", low_memory=False)

In [ ]:
X_full = dataset_pipeline.fit_transform(raw_df)
feature_names = X_full.columns
X_full = X_full.values

y_full = TargetPreprocessor().fit_transform(raw_df)

## Linear model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
def crps_exact(y_true, y_pred):
    """CRPS when y_true and y_pred are given as exact values."""
    return np.abs(y_true - y_pred) / 199

In [ ]:
class LinearModel(BaseEstimator):
    def __init__(self, **lr_params):
        self.lr = None
        self.lr_params = lr_params
    
    def fit(self, X, y):
        self.lr = LinearRegression(**self.lr_params)
        self.lr.fit(X, y)
        return self
    
    def predict(self, X):
        return np.clip(np.round(self.lr.predict(X)), -99, 99)
    
    def score(self, X, y):
        y_pred = self.predict(X)
        # NB. Higher scores are better, but lower CRPS is better
        return -np.mean(crps_exact(y, y_pred))

## Feature selection

In [ ]:
from sklearn.model_selection import ShuffleSplit
from rwfs import RandomWalkFeatureSelection

cv_nfl = ShuffleSplit(n_splits=10, test_size=0.5, random_state=42)
model_nfl = LinearModel()
rwfs_nfl = RandomWalkFeatureSelection(model_nfl, cv_nfl, n_steps=1000, 
                                     initial_fraction=0.5,
                                     temperature=1e-4, cooldown_factor=0.99,
                                     agg=np.mean,
                                     cache_scores=False)

In [ ]:
%%time
rwfs_nfl.fit(X_full, y_full, verbose=0)

## Evaluation

In [ ]:
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import pandas as pd

In [ ]:
diagnostics = pd.DataFrame(rwfs_nfl.diagnostics_)
plt.plot(-diagnostics['score'], 'g')
plt.plot(-diagnostics['score'] + diagnostics['se'], 'g--')
plt.plot(-diagnostics['score'] - diagnostics['se'], 'g--')
plt.plot(-diagnostics['mean_score'], 'r', alpha=0.5)
#plt.ylim(0.0, 1.0)
plt.title(f"Minimum CRPS found: {rwfs_nfl.best_score_:.5f}")
plt.show()

There is quite a lot of variance visible in the CV scores, but at least the mean score is decreasing nicely. There is an option for caching evaluations of feature sets, but by reevaluating at each step, there should be be some extra robustness.

Let's look at the features found in the best-scoring feature set (not necessarily the absolutely most important, we'll look at feature importances later):

In [ ]:
feature_names[list(rwfs_nfl.best_features_)]

Both `distance` and `defenders_in_the_box` are features I would expect to see here.  The 4th down has quite a bit of different distribution than the other downs, so that also makes sense.

I'm not so sure about the formations, would probably have to check how many observations we have for those, and quarter 4 is a bit of a surprise to me.

Now, it is very much possible that some less important features might show up in the best-scoring feature set; eg., a linear regressor might give some features very low coefficients whereby they wouldn't affect the score much, even if they are included.

Instead, let's look at how often a given feature is included in an accepted state.  That gives us an idea of just how essential it is.

`rwfs.feature_importances_` gives us this by computing the fraction of accepted stats the features are part of. By default, it isues a exponentially weighted mean giving more weight to later observations, whcih can be tweaked by the parameter `gamma` (`gamma=1.0` takes a flat average of all observations).

In [ ]:
importances = rwfs_nfl.feature_importances_
plt.bar(feature_names, importances)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
feature_names[rwfs_nfl.feature_importances_ > 0.8]

And wee see that `defenders_in_the_box`, `distance`, and `quarter_4` retain their status of being the most important when we pick the features with more than 0.9 feature importance.

Different runs can of course produce different results, but truly important features should still show up if we take enough steps and tune our parameters appropriately.  The temperature is highly dependent on the scale of the scores, and you may want to decrease the cooldown for longer runs.